# Homology of a Dowker complex

In this example we

- compute the homology of a Dowker complex
- print summary statistics (betti numbers, dimensions of cycle and boundary spaces)
- plot 1d and 2d cycle representatives

for two examples: a **circle** represented as the suspension of two points, and a **sphere** represented by the suspension of the circle

In [ ]:
import oat_python as oat

import plotly.graph_objects as go
import numpy as np

# Define a circle

As a cycle graph with four edges

In [ ]:
circle                  =   dict( A= ["i","j"], B=["j","k"], C=["k","l"], D=["i","l"] )

# Reformat the dictionary as a list of sorted-lists of integers

In [ ]:
# relabel
listlist, translator    =   oat.hypergraph.relabel( circle )

# display the results of relabeling
display("list of lists format", listlist)
display("relabeling dicionary", translator)

# run a simple check to verify that relabeling worked correctly
for counter, hyperedge in enumerate(listlist):
    [ translator["new_node_to_old_node"][node] for node in hyperedge  ]
    circle[ translator["new_edge_to_old_edge"][counter] ]
    if not [ translator["new_node_to_old_node"][node] for node in hyperedge  ] == circle[ translator["new_edge_to_old_edge"][counter] ]:
        raise ValueError('The relabeled hypergraph should map perfectly onto the initial hypergraph.')

# Compute homology

By factoring the boundary matrix of the Dowker complex.

In [ ]:
factored    =   oat.rust.FactoredBoundaryMatrixDowker( dowker_simplices = listlist, max_homology_dimension =2  )
homology    =   factored.homology()

print( "\n===================================================================================")
print(    "Cycle representatives that represent a basis for homology in dimensions 0, 1, and 2")
print(    "===================================================================================")
display( homology )
print(  "\n===============================")
print(    "Individual cycle representative")
print(    "===============================")
homology["cycle representative"][1]

# Plot a cycle representative with multidimensional scaling

In [ ]:
#   EXTRACT A LIST OF TRIANGLE WHERE THE CYCLE TAKES NONZERO COEFFICIENTS
#   ---------------------------------------------------------------------

cycle           =   homology["cycle representative"][1]
edges           =   cycle["simplex"].tolist()

#   GENERATE MDS COORDINATES FOR EACH VERTEX, BASED ON "HOP DISTANCE" WITHIN
#   THE GRAPH COMPOSED OF EDGES THAT ARE INCIDENT TO TRIANGLES IN THE CYCLE
#   ------------------------------------------------------------------------

coo             =   oat.plot.hop_mds_from_simplices( edges, dimension=2 ) # coo stands for "coordinate oracle"
x               =   [pt[0] for pt in coo.values()]
y               =   [pt[1] for pt in coo.values()]

#   GENERATE A TRACE FOR THE POINT CLOUD
#   ------------------------------------

data            =   []
trace           =   go.Scatter(
                        x=x,y=y,  # x, y, z coordinates
                        mode="markers+text", # indicates we want some text to appear next to each marker
                        text=[f"Node " + old_node for old_node in  translator["new_node_to_old_node"] ], # the text we want to appear next to each point
                        textposition="top center", # where we want the text positioned, relative to the marker
                        name="Nodes",
                    )
data.append(trace)

#   GENERATE A TRACE FOR EACH TRIANGLE
#   ----------------------------------

for edge in edges:

    # get the name or the original edge
    for (p,lis) in enumerate(listlist):
        if edge == lis:
            edge_name = translator["new_edge_to_old_edge"][p]    

    # get the original vertex labels of the edge
    edge_vertices = [ translator["new_node_to_old_node"][p] for p in edge ]
    
    # generate a trace, using the old edge name
    trace       =   oat.plot.edge__trace2d(edge=edge, coo=coo)
    trace.update(
        line=dict( color="red" ), # let's color the edge red 
        opacity=0.5, # sets the transparency
        showlegend=True, # indicate we want this simplex to appear in the legend
        name= f"Simplex {edge_name}", # label in the legend entry
        text=f"Vertices: {edge_vertices}", # text we want to appear when hovering the cursor over the simple
        )
    data.append(trace)

#   ADJUST THE PLOT LAYOUT
#   ----------------------

fig             =   go.Figure(data)
fig.update_layout(
    title="Hover cursor over a simplex to show its list of vertices<br>Click legend entries to toggle on/off",
    width=1000, 
    height=1000,     
    scene = dict(
        aspectratio=go.layout.scene.Aspectratio(x=1, y=1, z=1), # controls zoom
        xaxis = dict(range=[-1, 1],), # x axis limits
        yaxis = dict(range=[-1, 1],), # y axis limits
        zaxis = dict(range=[-1, 1],), # z axis limits
    ),    
)
fig.show()